In [1]:
from collections import OrderedDict

# Base descriptor class for type enforcement
class Typed:
    _expected_type = type(None)

    def __init__(self, name=None):
        self._name = name

    def __set__(self, instance, value):
        if not isinstance(value, self._expected_type):
            raise TypeError(f"Expected {self._expected_type}")
        instance.__dict__[self._name] = value

# Specific type descriptors
class Integer(Typed):
    _expected_type = int

class Float(Typed):
    _expected_type = float

class String(Typed):
    _expected_type = str

# Metaclass to capture definition order
class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        order = [name for name in clsdict if isinstance(clsdict[name], Typed)]
        for name in order:
            clsdict[name]._name = name
        d['_order'] = order
        return type.__new__(cls, clsname, bases, d)

    @classmethod
    def __prepare__(cls, clsname, bases):
        return OrderedDict()

# Base structure class using the metaclass
class Structure(metaclass=OrderedMeta):
    def as_csv(self):
        return ','.join(str(getattr(self, name)) for name in self._order)

# Example usage
class Stock(Structure):
    name = String()
    shares = Integer()
    price = Float()

    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

# Testing the implementation
s = Stock("GOOG", 100, 490.1)
print(s.as_csv())  # Output: GOOG,100,490.1

# Attempting to set an incorrect type
try:
    t = Stock("AAPL", "a lot", 610.23)
except TypeError as e:
    print(e)  # Output: Expected <class 'int'>


GOOG,100,490.1
Expected <class 'int'>
